In [10]:
from simple.binance import getHistMonth
from simple.plotly import chartFigure, interactFigure
import pandas as pd
from simple.pretty import pmap
from calendar import day_name

In [2]:
frame = '1h'
tickers = ['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'XRPUSDT']

In [3]:
def getDiff(ticker):
    return getHistMonth('2019-01', '2022-05', ticker, frame, close_only=True).diff().dropna()

In [4]:
F = pd.concat(pmap(getDiff, tickers), axis=1)
F = F.join(F.index.isocalendar())
F['week_hour'] = F.groupby(['year', 'week']).cumcount()
F

  0%|          | 0/6 [00:00<?, ?it/s]

,BTCUSDT,ETHUSDT,LTCUSDT,TRXUSDT,ETCUSDT,XRPUSDT,year,week,day,week_hour
DT,,,,,,,,,,
2019-01-01 01:00:00,-10.62,-0.68,-0.22,-0.00013,-0.0261,-0.00109,2019,1,2,0
2019-01-01 02:00:00,0.31,-0.42,-0.02,0.00002,-0.0243,-0.00168,2019,1,2,1
2019-01-01 03:00:00,3.13,0.60,0.06,0.00014,0.0465,0.00234,2019,1,2,2
2019-01-01 04:00:00,-0.42,1.03,0.14,0.00013,0.0126,0.00179,2019,1,2,3
2019-01-01 05:00:00,7.23,0.40,0.09,0.00011,0.0528,0.00081,2019,1,2,4
...,...,...,...,...,...,...,...,...,...,...
2022-05-31 19:00:00,-57.20,-0.33,0.40,0.00052,0.1700,0.00120,2022,22,2,43
2022-05-31 20:00:00,187.94,19.45,0.60,0.00100,0.1600,0.00550,2022,22,2,44
2022-05-31 21:00:00,-93.64,-6.20,-0.20,-0.00023,-0.0300,0.00100,2022,22,2,45


In [5]:
P = F.groupby('week_hour')['BTCUSDT'].mean().cumsum()
P

week_hour
0       25.794134
1       41.340391
2        9.267207
3       -1.238883
4       -8.014693
          ...    
163     42.744362
164    100.803480
165    150.882888
166    160.179737
167    162.755049
Name: BTCUSDT, Length: 168, dtype: float64

In [6]:
Hours = list(range(0, 168, 24))
Hours

[0, 24, 48, 72, 96, 120, 144]

In [9]:
def model(index: int = (0, len(tickers))):
    Profile = F.groupby('week_hour')[tickers[index]].mean().cumsum()
    return 0, locals()

In [7]:
fig = chartFigure(height=500,
                  Profile=dict(y=P, width=6, opacity=0.3),
                  Weekdays=dict(x=Hours, y=[0] * 7, mode="text", textposition="top right",
                                text=[f'{day: >16}' for day in day_name]))

Hours.append(168)
[fig.add_vline(x=hour, line_color='gray', line_dash='dot') for hour in Hours]

fig

FigureWidgetResampler({
    'data': [{'line': {'width': 6},
              'name': 'Profile',
              'op…